# AutoGluon Binary

In [2]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
binaryDf = TabularDataset('binary_dataset.csv')

/Users/axelsariel/repos/HousingDemographics/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
train_data, test_data = train_test_split(binaryDf, test_size=0.25, random_state=0)
label = 'GrowthRate'
train_data[label].value_counts(normalize=True)

GrowthRate
 1.0    0.862346
 0.0    0.071063
-1.0    0.066591
Name: proportion, dtype: float64

In [4]:
predictor = TabularPredictor(label=label).fit(train_data, time_limit=300)

No path specified. Models will be saved in: "AutogluonModels/ag-20240429_191124"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels/ag-20240429_191124"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.6
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 23.0.0: Fri S

In [6]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

11553     1.0
1177      1.0
3111      1.0
111294    1.0
173138    1.0
Name: GrowthRate, dtype: float64

In [7]:
y_pred.describe()

count    62532.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: GrowthRate, dtype: float64

In [8]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.8622945052133307,
 'balanced_accuracy': 0.3333333333333333,
 'mcc': 0.0}

In [1]:
predictor.leaderboard(test_data)

NameError: name 'predictor' is not defined

In [ ]:
predictor.feature_importance(train_data)